In [1]:
import pandas as pd
import os
import matplotlib
import matplotlib.pyplot as plt
import requests
from datetime import datetime, timedelta
from pandas.plotting import register_matplotlib_converters
from htmldate import find_date
from util.api import get_lts_list
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
import re
%matplotlib inline
plt.style.use('classic')

In [2]:
lts_list = get_lts_list()
df_list = []
df_by_category_list = []

for idx, lts in enumerate(lts_list):
    df = pd.read_pickle(f'../../data/lts-{lts}/lts-{lts}.df')
    df_list.append(df)

In this section, we respond the following two questions:
​
- How are packages distributed in stackage?
- How are packages using mtl distributed in Stackage?
​
For this, we make the following graphs
​
1. Line graph that shows the number of packets for each published snapshot over time
2. Bar graph that shows the evolution of the main categories of the packages
3. Bar graph that shows the evolution of the stabilities of the packages
4. Bar graph that shows the evolution of the main categories of the packages that depend of `mtl`
5. Bar graph that shows the evolution of the stabilities of the packages that depend of `mtl`
​
## RQ0. 1 - Line graph that shows the number of packets for each published snapshot over time

In [75]:

total_downloads = 0
for idx,row in df_list[-1].iterrows():
    # Make a request
    url = "https://hackage.haskell.org/package/%s" % row["package"]
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    downloads = 0
    # Extract head of page
    try:
        downloads = 0
        datas = soup.find(id = 'properties')
        downloads = datas.find("th", string="Downloads").parent.select("td")
        downloads = str(downloads[0]).replace("<td>", "").split(" ",1)
        total_downloads += int(downloads[0])
    except:
        print(row["package"])
        print('----------------------')
    # print the result
print("Total de descargas en la lts",total_downloads)

80635417


In [5]:
import re
def read_lines_code(fullpath):
    line_of_code = 0
    still_comment = 0
    file = open(fullpath,'r',encoding='latin-1')
    for idx,line in enumerate(file):
        no_space = line.strip()
        if no_space:
            
            big_comment = re.match("^({-[^#]*$)", no_space)
            if big_comment:
                still_comment += 1
                
            end_big_comment = re.match("[^#]*-}$", no_space)
            if end_big_comment:
                still_comment -= 1
                continue
            
            if still_comment != 0:
                continue
               
            comment = re.match("^\--+", no_space)
            if not comment and still_comment == 0:
                line_of_code += 1
                
    return line_of_code

In [6]:
import math

def convert_size(size_bytes):
    if size_bytes == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
    i = int(math.floor(math.log(size_bytes, 1024)))
    p = math.pow(1024, i)
    s = round(size_bytes / p, 2)
    return "%s %s" % (s, size_name[i])

In [10]:
list_size = []
list_sizeMB = []
for idx,lts in enumerate(lts_list):
    size = 0
    sizeMB = 0
    folderpath = f'C:/Users/nicol/Desktop/lts/lts-{lts}'
    
    for path, dirs, files in os.walk(folderpath):
        for f in files:
            fp = os.path.join(path, f)
            sizeMB += os.stat(fp).st_size
            if f.lower().endswith('.hs'):
                try:
                    size += read_lines_code(fp)
                except Exception as e:
                    print('File ',fp,' with error: ',e)
    #Transform to MegaBytes          
    print("Folder LTS-" +str(lts)+ " size: " + str(size)+" LOC" + str(convert_size(sizeMB)))
    list_size.append(size)
    list_sizeMB.append(sizeMB)
print("Las lts descargadas pesan: ",str(convert_size(sum(list_sizeMB))))

Folder LTS-0-7 size: 0 LOC0B
Folder LTS-2-22 size: 0 LOC0B
Folder LTS-3-22 size: 0 LOC0B
Folder LTS-6-35 size: 0 LOC0B
Folder LTS-7-24 size: 0 LOC0B
Folder LTS-9-21 size: 0 LOC0B
Folder LTS-11-22 size: 0 LOC0B
Folder LTS-12-14 size: 0 LOC0B
Folder LTS-12-26 size: 0 LOC0B
Folder LTS-13-11 size: 0 LOC0B
Folder LTS-13-19 size: 0 LOC0B
Folder LTS-14-27 size: 0 LOC0B
Folder LTS-15-3 size: 0 LOC0B
Folder LTS-16-11 size: 0 LOC0B
Folder LTS-16-31 size: 0 LOC0B
Folder LTS-17-2 size: 0 LOC0B
Folder LTS-18-6 size: 0 LOC0B
Folder LTS-18-8 size: 0 LOC0B
Folder LTS-18-18 size: 0 LOC0B
Las lts descargadas pesan:  0B


In [ ]:
total_pkgs = [len(df) for df in df_list]
lts_datetim=[]
for lts in lts_list:
    url = f"https://www.stackage.org/lts-{lts.replace('-','.')}"
    response = requests.get(url)
    my_date = find_date(response.content, extensive_search=True)
    date_time_obj = datetime.fromisoformat(my_date)
    lts_datetim.append(date_time_obj)
    
lts_date = ["2014-12"]
dates = [datetime(2014,12,1)]
year = 2015
for i in range (7):
    month = 0
    for i in range (4):
        month += 3
        date = str(year)+"-"+str(month)
        formato = datetime(year,month,1)
        lts_date.append(date)
        dates.append(formato)
    year += 1
    
dates = matplotlib.dates.date2num(dates)

In [ ]:
register_matplotlib_converters()

fig, ax = plt.subplots(figsize=(18,6))
ax.margins(0.015) 
plt.subplots_adjust(right=1.1)


#line = ax.plot(dates, total_pkgs, color='grey', marker='o',label='lts version')
lns1 = ax.plot_date(lts_datetim, total_pkgs, linestyle='solid',color='black',label='LTS release')
plt.gcf().autofmt_xdate()

for i in range(len(lts_list)):
    if i == 3 or i == 17 or i == 14:
        plt.text(lts_datetim[i],total_pkgs[i]-280,lts_list[i],ha="center",rotation = -35)
    elif i== 12:
        plt.text(lts_datetim[i],total_pkgs[i]-200,lts_list[i],ha="center",rotation = -35)
    else:
        plt.text(lts_datetim[i],total_pkgs[i],lts_list[i],ha="center",rotation = 35)
    
ax2 = ax.twinx()
lns2 = ax2.plot_date(lts_datetim, list_size, linestyle='--',color='grey',label='LOC')

ax.set_yticks([0,500, 1000, 1500, 2000, 2500, 3000])
ax.set_yticklabels(['0','500', '1000', '1500', '2000', '2500', '3000'],fontsize=15)

ax2.set_yticks([0, 500000, 1000000, 1500000, 2000000, 2500000, 3000000, 3500000,4000000,4500000,5000000])
ax2.set_yticklabels(['0', '0.5 Mega','1.0 Mega', '1.5 Mega', '2.0 Mega', '2.5 Mega', '3.0 Mega', '3.5 Mega', '4.0 Mega', '4.5 Mega','5.0 Mega'],fontsize=15)

ax.set_xticks(dates)
ax.set_xticklabels(lts_date,fontsize=15)

ax.set_xlabel('Published date',fontsize=18)
ax.set_ylabel('Number of packages',fontsize=18)
ax2.set_ylabel('Lines of code',fontsize=18)

lns = lns1+lns2
labs = [l.get_label() for l in lns]
ax.legend(lns, labs, loc='upper left')

plt.savefig("../../graphs/rq0-1.pdf", format='pdf', bbox_inches='tight')
plt.show()